In [ ]:
!apt-get update -y
!apt-get install -y \
libglib2.0-0 \
libnss3 \
libfontconfig1 \
gconf2-common \
libgbm1 \
udev \
libu2f-udev
!apt --fix-broken install -y
!wget -P /tmp https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/119.0.6045.105/linux64/chrome-linux64.zip
!unzip /tmp/chrome-linux64.zip -d /usr/bin/
!wget -P /tmp https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/119.0.6045.105/linux64/chromedriver-linux64.zip
!unzip /tmp/chromedriver-linux64.zip -d /usr/bin/
!apt install -y python3-selenium
!pip install selenium==4.7.0
!pip install webdriver_manager

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [60.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,545 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Package

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from lxml import etree

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
CHROME_BINARY_LOCATION = "/usr/bin/chrome-linux64/chrome"
CHROMEDRIVER_BINARY_LOCATION = "/usr/bin/chromedriver-linux64/chromedriver"

def add_driver_options(options):
    """
    Add configurable options
    """
    chrome_options = Options()
    for opt in options:
        chrome_options.add_argument(opt)
    return chrome_options

def initialize_driver():
    """
    Initialize the web driver
    """
    driver_config = {
        "options": [
            "--headless",
            "--no-sandbox",
            "--allow-insecure-localhost",
            "--disable-dev-shm-usage",
            # "--incognito",
            "--window-size=1920x1080",
            "user-agent=Chrome/119.0.6045.105"
        ],
    }
    options = add_driver_options(driver_config["options"])
    options.binary_location = CHROME_BINARY_LOCATION
    driver = webdriver.Chrome(
        executable_path=CHROMEDRIVER_BINARY_LOCATION,
        options=options)
    return driver
def scroll_to_bottom(driver, pause_time=2):
    """
    Cuộn trang xuống đáy để tải nội dung động.
    """
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #time.sleep(pause_time)
        WebDriverWait(driver, pause_time).until(lambda driver: driver.execute_script("return document.body.scrollHeight") >= last_height)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
def extract_google_maps_link(driver):
    try:

        time.sleep(5)


        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')


        google_maps_links = [
            a['href'] for a in soup.find_all('a', href=True)
            if "maps.google.com" in a['href']
        ]


        if not google_maps_links:
            print("Checking in Shadow DOM...")
            google_maps_links = driver.execute_script("""
                let links = Array.from(document.querySelectorAll('a[href*="maps.google.com"]'));
                return links.map(link => link.href);
            """)

        if google_maps_links:
            print("Google Maps Links Found:", google_maps_links)
        else:
            print("No Google Maps Links Found.")
        return google_maps_links

    except Exception as e:
        print(f"Error extracting Google Maps links: {e}")
        return []
def check_js_loaded(driver):
    try:
        element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "hpipapi atm_7l_1kw7nm4 atm_c8_1x4eueo atm_cs_1kw7nm4 atm_g3_1kw7nm4 atm_gi_idpfg4 atm_l8_idpfg4 atm_kd_idpfg4_pfnrn2 dir dir-ltr"))
        )
        print("JavaScript đã được load và phần tử xuất hiện!")
    except:
        print("Không thể tìm thấy phần tử hoặc JavaScript chưa load.")
    is_js_loaded = driver.execute_script("return document.readyState") == "complete"

    if is_js_loaded:
        print("Trang đã hoàn tất load JavaScript.")
    else:
        print("Trang chưa hoàn tất load JavaScript.")

    return 0

In [ ]:
import json
import os
import re
import time
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from bs4 import BeautifulSoup
from lxml import etree
import requests
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
# from break_captcha import *
# from driver_init import *


def next_day(date,date_format = "%Y-%m-%d", days=1):
    date = datetime.strptime(date, date_format)
    next_day = date + timedelta(days=days)
    return next_day.strftime(date_format)
def next_month(date, date_format="%Y-%m-%d", months=1):
    date = datetime.strptime(date, date_format)
    next_month = date + relativedelta(months=months)
    return next_month.strftime(date_format)
class Crawler:
    def __init__(self):
        self.home_page = f"https://www.airbnb.com.vn/s/H%C3%A0-N%E1%BB%99i--Vietnam/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date={start_date}&monthly_length=0&monthly_end_date={end_date}&price_filter_input_type=0&channel=EXPLORE&search_type=filter_change&price_filter_num_nights=211&query=H%C3%A0%20N%E1%BB%99i%2C%20Vietnam&place_id=ChIJKQqAE44ANTERDbkQYkF-mAI&date_picker_type=calendar&checkin={check_in}&checkout={check_out}&source=structured_search_input_header&search_mode=regular_search"
        self.root = "https://www.airbnb.com.vn"
        self.head_page = [self.home_page]


    def get_pages(self, url):

      bs = BeautifulSoup(self.driver.page_source, 'html.parser')
      elements = bs.find_all('a', class_ = 'rfexzly atm_9s_1ulexfb atm_7l_1j28jx2 atm_e2_1osqo2v dir dir-ltr')
      data = []

      try:
          # for feature in elements:
          #     feature = feature.find_all('a', href=True)
          #     data.append(feature[0]['href'])
          for feature in elements:
            q = feature.get('href')
            if q not in data:
              data.append(q)
            print(q)
      except Exception as e:
            print(e)

      pages = bs.find_all('a', class_= 'l1ovpqvx atm_1he2i46_1k8pnbi_10saat9 atm_yxpdqi_1pv6nv4_10saat9 atm_1a0hdzc_w1h1e8_10saat9 atm_2bu6ew_929bqk_10saat9 atm_12oyo1u_73u7pn_10saat9 atm_fiaz40_1etamxe_10saat9 c1ackr0h atm_c8_km0zk7 atm_g3_18khvle atm_fr_1m9t47k atm_cs_10d11i2 atm_9s_1txwivl atm_h_1h6ojuz atm_fc_1h6ojuz atm_bb_idpfg4 atm_3f_glywfm atm_5j_1ssbidh atm_26_1j28jx2 atm_7l_jt7fhx atm_vy_1vi7ecw atm_e2_1vi7ecw atm_gi_idpfg4 atm_gz_1yuitx atm_h0_1yuitx atm_l8_idpfg4 atm_uc_10d7vwn atm_kd_glywfm atm_uc_glywfm__1rrf6b5 atm_26_zbnr2t_1nos8r_uv4tnr atm_tr_kv3y6q_csw3t1 atm_26_zbnr2t_csw3t1 atm_9j_73adwj_1o5j5ji atm_3f_glywfm_jo46a5 atm_l8_idpfg4_jo46a5 atm_gi_idpfg4_jo46a5 atm_3f_glywfm_1icshfk atm_kd_glywfm_19774hq atm_uc_aaiy6o_1w3cfyq atm_26_zbnr2t_1w3cfyq atm_70_1agwrbf_1w3cfyq atm_uc_glywfm_1w3cfyq_1rrf6b5 atm_uc_aaiy6o_pfnrn2_1oszvuo atm_26_zbnr2t_pfnrn2_1oszvuo atm_70_1agwrbf_pfnrn2_1oszvuo atm_uc_glywfm_pfnrn2_1o31aam dir dir-ltr')

      print('start')
      try:
          # for feature in elements:
          #     feature = feature.find_all('a', href=True)
          #     data.append(feature[0]['href'])
          if not pages:
              print("No pages found.")
              return
          for feature in pages:
            q = feature.get('href')
            link = 'https://www.airbnb.com.vn'+q
            if link not in self.head_page:
              self.head_page.append(link)

              print(q)
      except Exception as e:
            print(e)
      print(len(self.head_page))
      return [self.root + link for link in data]

    def extract_lat_lon(self, url):
        attempts = 0
        success = False

        while not success and attempts < 10:
            try:
                r = requests.get(url)
                p_lat = re.compile(r'"lat":([-0-9.]+),')
                p_lon = re.compile(r'"lng":([-0-9.]+),')
                lat = p_lat.findall(r.text)[0]
                lon = p_lon.findall(r.text)[0]
                success = True
                return lat , lon
            except:
                print('delete this row')
                attempts += 1
        return 0 , 0

    def retrieve_data(self, page_source, page_link):

        data = {}
        bs = BeautifulSoup(page_source)
        page = etree.HTML(str(bs))
        lat, lon = self.extract_lat_lon(page_link)
        print(lat, lon)
        data['lat'] = lat
        data['lon'] = lon
        try:
            name_element = page.xpath("""//*[@class="hpipapi atm_7l_1kw7nm4 atm_c8_1x4eueo atm_cs_1kw7nm4 atm_g3_1kw7nm4 atm_gi_idpfg4 atm_l8_idpfg4 atm_kd_idpfg4_pfnrn2 i1pmzyw7 atm_9s_1nu9bjl dir dir-ltr"]""")
            if name_element:
                name = name_element[0].text.strip()
                data['name'] = name
            else:
                data['name'] = "N/A"

        except Exception as e:
            print(f"Error retrieving name: {e}")
            data['name'] = "N/A"
            data['name'] = name
        print(data['name'])
        data['check_in'] = check_in
        data['check_out'] = check_out
        elements = bs.find_all(class_='l7n4lsf atm_9s_1o8liyq_keqd55 dir dir-ltr')
        data_1 = ""
        for element in elements:
            data_1 += element.text.strip()
        data['room_info'] = data_1
        price = page.xpath("""//*[@class="_11jcbg2"]""")[0].text.strip()
        data['price_1_day'] = price

        elements2 = bs.find_all(class_='iikjzje atm_9s_1txwivl atm_h_1h6ojuz atm_le_exct8b atm_j3_e9shpx__oggzyc i4wvyiy atm_fc_esu3gu atm_ar_1sbvcyy dir dir-ltr')
        if elements2:

            for element in elements2:
                div_2 = element.find_all('div')[0]
                feature = div_2.text.strip()
                if feature.startswith('Không'):
                  feature = feature.replace('Không có: ', '').strip()
                  data[feature] =-1
                else:
                  data[feature] =1


        elements3 = bs.find_all(class_='vjb6p42 atm_c8_vvn7el atm_g3_k2d186 atm_fr_1vi102y dir dir-ltr')
        if elements3:
            rate = ['cleanliness', 'Accuracy', 'Check in', 'Communicate', 'Location', 'Value']
            x =0
            for element in elements3:
                feature = element.text.strip()
                data[f'rating {rate[x]}'] = feature
                x +=1
        elements4 = bs.find_all(class_='tijjzz2 atm_7l_dezgoh atm_bx_48h72j atm_cs_10d11i2 atm_c8_11rlvjh atm_g3_1xap3gc atm_fr_1xai86l atm_9s_116y0ak atm_h_1h6ojuz dir dir-ltr')
        if elements4:
            for element in elements4:
                review_text_element = element.find('div', dir='ltr')

                if review_text_element:
                    review_text = review_text_element.text.strip()
                    data['total_reviews'] = review_text
                else:
                    data['total_reviews'] = "less thanh 3 reviews"

        return data

    def prepare_driver(self):
        self.driver = initialize_driver()

    def run(self, num_pages):

        main_url = self.head_page[num_pages]

        if os.path.exists("data/index.json"):  # index
            with open("data/index.json") as f:
                index = set(json.load(f))
        else:
            index = set()
        global dataset_15_page
        global dataset
        try:
            dataset_1_page = []
            self.driver.get(main_url)
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            print(self.driver.current_url)
            time.sleep(15)
            list_link_pages = self.get_pages(main_url)
            print(len(list_link_pages), 'pages')
            for page_link in list_link_pages:
                self.driver.get(page_link)
                time.sleep(5)
                data = self.retrieve_data(self.driver.page_source, page_link)
                print(data)
                dataset_1_page.append(data)
                dataset_15_page.append(data)
                dataset.append(data)
                index.add(data['name'])
                self.driver.delete_all_cookies()
            # dataset_15_page.extend(dataset_1_page)
            # dataset.extend(dataset_1_page)
            if len(dataset_1_page) != 0:
                print(len(dataset_1_page), 'test')
                df = pd.DataFrame(dataset_1_page)
                df.to_csv(f"data/page_{num_pages}_checkin_{check_in}.csv", index=False, encoding='utf-8-sig')

            with open("data/index.json", "w") as f:
                json.dump(list(index), f)
            self.driver.delete_all_cookies()
            # self.driver.switch_to.window(self.driver.window_handles[0]); self.driver.close(); self.driver.switch_to.window(self.driver.window_handles[1])
        except Exception as e:
            print(e)

In [ ]:
import time
import os


if __name__ == "__main__":
    start_date ='2025-10-01'
    end_date = '2026-02-01'
    check_in ='2025-11-01'
    check_out ='2025-11-07'
    folder_name = 'data'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    dataset = []
    for i in range(3):

      crawler = Crawler()
      crawler.prepare_driver()
      dataset_15_page = []
      for i in range(0, 16):
        if i != 4: # hoặc 5
            start_time = time.time()
            crawler.run(i)
            end_time = time.time()
            execution_time = end_time - start_time
            print(f"Execution time: {execution_time} seconds")

      df_15_page = pd.DataFrame(dataset_15_page)
      df_15_page.to_csv(f"data/checkin_{check_in}.csv", index=False, encoding='utf-8-sig')


      # check_in = next_day(check_in, days=30)
      # check_out = next_day(check_out, days=30)
      check_in = next_month(check_in, months=1)
      check_out = next_month(check_out, months=1)
    df_all = pd.DataFrame(dataset)
    df_all.to_csv(f"data/full.csv", index=False, encoding='utf-8-sig')

<ipython-input-4-a00837588b6a>:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


Streaming output truncated to the last 5000 lines.
/rooms/1202695250684669697?search_mode=regular_search&check_in=2025-12-01&check_out=2025-12-07&source_impression_id=p3_1734430986_P3Y8uFoDThvUi5Ut&previous_page_section_name=1000&federated_search_id=79bf6a3d-b1b0-4c47-96f7-9f4bb2aa5db9
/rooms/1202695250684669697?search_mode=regular_search&check_in=2025-12-01&check_out=2025-12-07&source_impression_id=p3_1734430986_P3Y8uFoDThvUi5Ut&previous_page_section_name=1000&federated_search_id=79bf6a3d-b1b0-4c47-96f7-9f4bb2aa5db9
/rooms/1202695250684669697?search_mode=regular_search&check_in=2025-12-01&check_out=2025-12-07&source_impression_id=p3_1734430986_P3Y8uFoDThvUi5Ut&previous_page_section_name=1000&federated_search_id=79bf6a3d-b1b0-4c47-96f7-9f4bb2aa5db9
/rooms/1202695250684669697?search_mode=regular_search&check_in=2025-12-01&check_out=2025-12-07&source_impression_id=p3_1734430986_P3Y8uFoDThvUi5Ut&previous_page_section_name=1000&federated_search_id=79bf6a3d-b1b0-4c47-96f7-9f4bb2aa5db9
/room